In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Hyper-parameters 
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)


In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
  
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


In [ ]:
for epoch in range(num_epochs):
  train(epoch)

Train Epoch: 0 [0/60000 (0%)]	Loss: 550.512617
Train Epoch: 0 [10000/60000 (17%)]	Loss: 180.459707
Train Epoch: 0 [20000/60000 (33%)]	Loss: 158.568877
Train Epoch: 0 [30000/60000 (50%)]	Loss: 150.201885
Train Epoch: 0 [40000/60000 (67%)]	Loss: 131.481816
Train Epoch: 0 [50000/60000 (83%)]	Loss: 128.658203
====> Epoch: 0 Average loss: 157.2139
Train Epoch: 1 [0/60000 (0%)]	Loss: 120.536104
Train Epoch: 1 [10000/60000 (17%)]	Loss: 124.261348
Train Epoch: 1 [20000/60000 (33%)]	Loss: 122.905039
Train Epoch: 1 [30000/60000 (50%)]	Loss: 114.546846
Train Epoch: 1 [40000/60000 (67%)]	Loss: 113.649141
Train Epoch: 1 [50000/60000 (83%)]	Loss: 116.990195
====> Epoch: 1 Average loss: 118.8996
Train Epoch: 2 [0/60000 (0%)]	Loss: 117.008066
Train Epoch: 2 [10000/60000 (17%)]	Loss: 118.054961
Train Epoch: 2 [20000/60000 (33%)]	Loss: 112.507539
Train Epoch: 2 [30000/60000 (50%)]	Loss: 110.427637
Train Epoch: 2 [40000/60000 (67%)]	Loss: 110.432354
Train Epoch: 2 [50000/60000 (83%)]	Loss: 109.024570
===

In [ ]:
dataset = datasets.MNIST(root='./data',transform=transforms.Compose([
                                                    transforms.Resize(30),
                                                    transforms.ToTensor()
                                           ]),download=True)
idx = dataset.targets==4
dataset.targets = dataset.targets[idx]
dataset.data = dataset.data[idx]
print(dataset.targets.shape)
loader = torch.utils.data.DataLoader(dataset=dataset,batch_size=5842)

torch.Size([5842])


In [ ]:
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 100)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), 'test'+str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


====> Test set loss: 101.6855
